In [ ]:
import json, os
import pandas as pd
from matplotlib import pyplot as plt
from collections import defaultdict
import numpy as np
from itertools import combinations
from sklearn.linear_model import LinearRegression, RANSACRegressor
from sklearn.decomposition import PCA
from wpca import WPCA
from sklearn.preprocessing import StandardScaler
from aquabyte.accuracy_metrics import AccuracyMetricsGenerator
from aquabyte.data_access_utils import S3AccessUtils, RDSAccessUtils
from aquabyte.optics import euclidean_distance, pixel2world
from aquabyte.visualize import Visualizer
import random
from scipy.stats import norm
from PIL import Image, ImageDraw
from urllib.parse import urlparse
from aquabyte.data_access_utils import S3AccessUtils, RDSAccessUtils

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_colwidth', 500)

In [ ]:
f = '/root/data/temp/results_c8cc936a26a7645895842f467fe9d88764cf5579_eide_data_lati_reconciled.h5'
df = pd.read_hdf(f, 'table')

In [ ]:
def get_centroid_depth(world_keypoints):
    depths = []
    for bp, wkp in world_keypoints.items():
        depths.append(wkp[1])
        
    return np.median(np.array(depths))

In [ ]:
plt.figure(figsize=(20, 10))
plt.hist(df.world_keypoints.apply(lambda x: get_centroid_depth(x)), bins=20)
plt.title('Spatial distribution of Eide lice valid fish')
plt.xlabel('Distance from Camera (m)')
plt.ylabel('Count')
plt.grid()
plt.show()

In [ ]:
f = '/root/data/temp/results_c8cc936a26a7645895842f467fe9d88764cf5579_kjeppevikholmen_data_lati_reconciled.h5'
df = pd.read_hdf(f, 'table')

plt.figure(figsize=(20, 10))
plt.hist(df.world_keypoints.apply(lambda x: get_centroid_depth(x)), bins=20)
plt.title('Spatial distribution of Kjeppevikholmen lice valid fish')
plt.xlabel('Distance from Camera (m)')
plt.ylabel('Count')
plt.grid()
plt.show()

In [ ]:
from aquabyte.data_access_utils import S3AccessUtils, RDSAccessUtils

In [ ]:
s3_access_utils = S3AccessUtils('/root/data')

In [ ]:
df['stereo_depth_m'] = df.world_keypoints.apply(lambda x: get_centroid_depth(x))

In [ ]:
import os
from urllib.parse import urlparse
FOCAL_LENGTH = 4015


def process_row(row, skip_reason):
    depth_m = row[depth_field]
    line_segment_length_px = object_length_m * FOCAL_LENGTH / depth_m
    image_url = row.image_url
    if 'aquabyte-crops-lati' not in image_url:
        bucket, key = 'aquabyte-crops', urlparse(image_url, allow_fragments=False).path.lstrip('/')
    else:
        components = urlparse(image_url, allow_fragments=False).path.lstrip('/').split('/')
        bucket, key = components[0], os.path.join(*components[1:])
    image_f = s3_access_utils.download_from_s3(bucket, key)

    im = Image.open(image_f)
    draw = ImageDraw.Draw(im)
    draw.line((100, 100, 100+line_segment_length_px, 100))

    f_name = os.path.basename(key)
    f = os.path.join(modified_images_dir, '{}_{}'.format(lo, hi), skip_reason, f_name)
    if not os.path.exists(os.path.dirname(f)):
        os.makedirs(os.path.dirname(f))
    im.save(f)


modified_images_dir = '/root/data/alok/lice_counting/blom_kjeppevikholmen_image_breakdown'
object_length_m = 0.01
N = 30

depth_values = [round(x, 1) for x in np.arange(0.2, 1.5, 0.1)]

depth_field = 'stereo_depth_m'
for i in range(len(depth_values)-1):
    print(i)
    lo, hi = depth_values[i], depth_values[i+1]
    depth_mask = (df[depth_field] >= lo) & (df[depth_field] <= hi)
    
    # accepted images
    for idx, row in df[depth_mask].head(N).iterrows():
        process_row(row, 'accepted')
    

In [ ]:
plt.scatter(list(range(df.shape[0])), df.sort_values('captured_at').centroid_depth)
plt.show()

In [ ]:
df.index = df.captured_at
df.id.resample('D', how=lambda x: x.shape[0])

In [ ]:
mask = (df.centroid_depth > 1.025) & (df.centroid_depth < 1.075)
df[mask].head()

In [ ]:
df.to_csv('/root/data/depth_values.csv')

In [ ]:
def fn(x):
    w, h = x['width'], x['height']
    theta = np.arctan(h / w) * (180.0 / np.pi)
    phi = np.arctan(0.25) * (180.0 / np.pi)
    if theta < phi:
        return w
    elif theta > 90.0 - phi:
        return h
    else:
        return (h**2 + w**2)**0.5

In [ ]:
df['image_width'] = df.metadata.apply(lambda x: x['width'])
df['image_height'] = df.metadata.apply(lambda x: x['height'])
df['length_px'] = df.metadata.apply(lambda x: fn(x))
df['estimated_depth'] = 4015 * 0.66 / df.length_px
# df['estimated_depth'] = 4015 * df.length / df.length_px

In [ ]:
plt.figure(figsize=(20, 10))
plt.scatter(df.centroid_depth, df.estimated_depth)
plt.plot([0.6, 1.5], [0.6, 1.5])
plt.xlabel('Ground truth distance from camera (m)')
plt.ylabel('Predicted distance from camera (m)')
plt.grid()
plt.show()

In [ ]:
df.estimated_depth.corr(df.centroid_depth)

In [ ]:
df.shape